In [19]:
import numpy as np
from numpy import concatenate
import matplotlib.pyplot as plt
import pandas as pd
import math
from math import sqrt
import tensorflow as tf
import sklearn
import seaborn as sns
from datetime import datetime
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from numpy import asarray
from sklearn.ensemble import RandomForestRegressor
from pandas import concat
from sklearn.model_selection import train_test_split

In [20]:
cof_df = pd.read_excel("coffee_data.xlsx")

In [21]:
cof_df

,Tên_mặt_hàng,Đơn_vị_tính,Loại_giá,Loại_tiền,Ngày,Giá
0,Cà phê Robusta nhân xô,VNĐ/kg,Thu mua,VNĐ,2021-01-01,31667.5
1,Cà phê Robusta nhân xô,Vnđ/kg,Thu mua,VNĐ,2021-01-04,32717.0
2,Cà phê Robusta nhân xô,Vnđ/kg,Thu mua,VNĐ,2021-01-05,32417.0
3,Cà phê Robusta nhân xô,Vnđ/kg,Thu mua,VNĐ,2021-01-06,32317.0
4,Cà phê Robusta nhân xô,Vnđ/kg,Thu mua,VNĐ,2021-01-07,32117.0
...,...,...,...,...,...,...
619,Cà phê Robusta nhân xô,Vnđ/kg,Thu mua,VNĐ,2023-05-15,55566.5
620,Cà phê Robusta nhân xô,Vnđ/kg,Thu mua,VNĐ,2023-05-16,56066.5
621,Cà phê Robusta nhân xô,Vnđ/kg,Thu mua,VNĐ,2023-05-17,56666.5
622,Cà phê Robusta nhân xô,Vnđ/kg,Thu mua,VNĐ,2023-05-18,57000.0


In [22]:
# Convert 'Ngày' column to datetime type
cof_df['Ngày'] = pd.to_datetime(cof_df['Ngày'])
# Sort the DataFrame by date in ascending order
cof_df.sort_values(by='Ngày', inplace=True)
cof_df['Giá Cà Phê'] =  cof_df['Giá']

cof_df = cof_df.drop(["Loại_tiền", "Loại_giá", "Tên_mặt_hàng", "Đơn_vị_tính", "Giá"], axis = 1)
cof_df

,Ngày,Giá Cà Phê
0,2021-01-01,31667.5
1,2021-01-04,32717.0
2,2021-01-05,32417.0
3,2021-01-06,32317.0
4,2021-01-07,32117.0
...,...,...
619,2023-05-15,55566.5
620,2023-05-16,56066.5
621,2023-05-17,56666.5
622,2023-05-18,57000.0


In [23]:
gas_df = pd.read_excel("fuel_data.xlsx")
gas_df

,Giá nhiên liệu,Xăng RON 95 1 lít,Xăng E5 RON 92 1 lít,Dầu Diesel 1 lít,Dầu Hỏa 1 lít
0,12.06.2023,22.01,20.87,18.02,17.82
1,01.06.2023,22.01,20.87,17.94,17.77
2,22.05.2023,21.49,20.48,17.95,17.96
3,11.05.2023,21.00,20.13,17.65,17.97
4,04.05.2023,22.32,21.43,18.25,18.52
...,...,...,...,...,...
114,02.03.2019,18.54,17.21,15.86,14.88
115,15.02.2019,17.60,16.27,14.90,14.18
116,31.01.2019,17.60,16.27,14.90,14.18
117,16.01.2019,17.60,16.27,14.90,14.18


In [24]:
gas_df["Ngày"] = pd.to_datetime(gas_df["Giá nhiên liệu"], format="%d.%m.%Y")
gas_df.drop(gas_df.columns[[0,1,2,4]], axis='columns', inplace=True)
gas_df

,Dầu Diesel 1 lít,Ngày
0,18.02,2023-06-12
1,17.94,2023-06-01
2,17.95,2023-05-22
3,17.65,2023-05-11
4,18.25,2023-05-04
...,...,...
114,15.86,2019-03-02
115,14.90,2019-02-15
116,14.90,2019-01-31
117,14.90,2019-01-16


In [25]:
# Merge the dataframes with an outer join
merged_df = pd.merge(cof_df, gas_df, left_on='Ngày', right_on='Ngày', how="outer")

# Fill missing dates from both datasets with each other
merged_df.fillna(method='bfill', inplace=True)

In [26]:
merged_df = merged_df.dropna()
merged_df

,Ngày,Giá Cà Phê,Dầu Diesel 1 lít
0,2021-01-01,31667.5,12.64
1,2021-01-04,32717.0,12.64
2,2021-01-05,32417.0,12.64
3,2021-01-06,32317.0,12.64
4,2021-01-07,32117.0,12.64
...,...,...,...
619,2023-05-15,55566.5,18.02
620,2023-05-16,56066.5,18.02
621,2023-05-17,56666.5,18.02
622,2023-05-18,57000.0,18.02


In [27]:
df = merged_df

In [ ]:
# Step 2: Data Exploration and Feature Engineering
# Convert 'Ngày' column to datetime type
df['Ngày'] = pd.to_datetime(df['Ngày'])

# 1. Date Features
df['DayOfWeek'] = df['Ngày'].dt.dayofweek
df['Month'] = df['Ngày'].dt.month
df['Year'] = df['Ngày'].dt.year

# 2. Lag Features
df['Prev_Coffee_Price'] = df['Giá Cà Phê'].shift(1)
df['Prev_Diesel_Price'] = df['Dầu Diesel 1 lít'].shift(1)

# 3. Rolling Window Statistics
window_size = 5
df['Rolling_Mean_Coffee'] = df['Giá Cà Phê'].rolling(window=window_size).mean()
df['Rolling_Std_Diesel'] = df['Dầu Diesel 1 lít'].rolling(window=window_size).std()

# 4. Price Difference
df['Coffee_Price_Diff'] = df['Giá Cà Phê'] - df['Prev_Coffee_Price']
df['Diesel_Price_Diff'] = df['Dầu Diesel 1 lít'] - df['Prev_Diesel_Price']

# 5. Interaction Terms
df['Interaction_Term'] = df['Giá Cà Phê'] * df['Dầu Diesel 1 lít']

# 6. Moving Averages
df['Moving_Avg_Coffee'] = df['Giá Cà Phê'].rolling(window=window_size).mean()
df['Moving_Avg_Diesel'] = df['Dầu Diesel 1 lít'].rolling(window=window_size).mean()

# Drop rows with NaN values resulting from the rolling window and lag computations
df.dropna(inplace=True)

# Re-split the data after feature engineering
X = df[['Dầu Diesel 1 lít', 'Prev_Diesel_Price', 'DayOfWeek', 'Month', 'Year', 'Rolling_Mean_Coffee',
        'Rolling_Std_Diesel', 'Coffee_Price_Diff', 'Diesel_Price_Diff', 'Interaction_Term',
        'Moving_Avg_Coffee', 'Moving_Avg_Diesel']]
y = df['Giá Cà Phê']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [28]:
# Step 3: Split the data
X = df[['Dầu Diesel 1 lít']]
y = df['Giá Cà Phê']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [29]:
# Step 4: Train the Random Forest Model
rf_model = RandomForestRegressor(random_state=42)
rf_model.fit(X_train, y_train)

RandomForestRegressor(random_state=42)

In [30]:
# Step 5: Extract the Feature from Random Forest Model
extracted_feature = rf_model.predict(X_train)


In [31]:
# Step 6: Data Preparation for RNN
X_train_rnn = np.column_stack((X_train, extracted_feature))


In [32]:
# Step 7: Build and Train the RNN Model
rnn_model = Sequential()
rnn_model.add(LSTM(50, activation='relu', input_shape=(2, 1)))
rnn_model.add(Dense(1))
rnn_model.compile(optimizer='adam', loss='mean_squared_error')

# Reshape X_train_rnn to (samples, timesteps, features) for LSTM input
X_train_rnn = X_train_rnn.reshape((X_train_rnn.shape[0], 2, 1))

rnn_model.fit(X_train_rnn, y_train, epochs=100, batch_size=16)


Epoch 1/100
32/32 [==============================] - 1s 1ms/step - loss: 1384527872.0000
Epoch 2/100
32/32 [==============================] - 0s 1ms/step - loss: 1198231040.0000
Epoch 3/100
32/32 [==============================] - 0s 1ms/step - loss: 939843840.0000
Epoch 4/100
32/32 [==============================] - 0s 1ms/step - loss: 571837888.0000
Epoch 5/100
32/32 [==============================] - 0s 1ms/step - loss: 158000368.0000
Epoch 6/100
32/32 [==============================] - 0s 1ms/step - loss: 4568083.5000
Epoch 7/100
32/32 [==============================] - 0s 1ms/step - loss: 809266.0000
Epoch 8/100
32/32 [==============================] - 0s 1ms/step - loss: 608816.2500
Epoch 9/100
32/32 [==============================] - 0s 1ms/step - loss: 599361.1250
Epoch 10/100
32/32 [==============================] - 0s 1ms/step - loss: 603221.7500
Epoch 11/100
32/32 [==============================] - 0s 1ms/step - loss: 605083.3750
Epoch 12/100
32/32 [=========================

32/32 [==============================] - 0s 1ms/step - loss: 608449.1250
Epoch 97/100
32/32 [==============================] - 0s 1ms/step - loss: 605609.5000
Epoch 98/100
32/32 [==============================] - 0s 1ms/step - loss: 614159.5000
Epoch 99/100
32/32 [==============================] - 0s 1ms/step - loss: 603934.3125
Epoch 100/100
32/32 [==============================] - 0s 1ms/step - loss: 604565.2500


In [33]:
# Step 8: Evaluate the RNN Model
X_test_rnn = np.column_stack((X_test, rf_model.predict(X_test)))
X_test_rnn = X_test_rnn.reshape((X_test_rnn.shape[0], 2, 1))
y_pred = rnn_model.predict(X_test_rnn)

mse = mean_squared_error(y_test, y_pred)
print(f'Mean Squared Error: {mse}')

4/4 [==============================] - 0s 1ms/step
Mean Squared Error: 363023.9705423279


In [34]:
# Step 9: Make Predictions
# Assuming you have new data with diesel price and you want to predict the coffee price
new_diesel_price = 18.02
new_feature = rf_model.predict(np.array([[new_diesel_price]]))

# Reshape new_feature to match the shape of the extracted_feature from training data
new_feature = new_feature.reshape((-1, 1))

# Concatenate the new_diesel_price and new_feature as input to RNN
new_input = np.concatenate((np.array([[new_diesel_price]]), new_feature), axis=1)

# Reshape new_input to (samples, timesteps, features) for LSTM input
new_input = new_input.reshape((1, 2, 1))

predicted_coffee_price = rnn_model.predict(new_input)[0][0]
print(f'Predicted Coffee Price: {predicted_coffee_price}')


1/1 [==============================] - 0s 15ms/step
Predicted Coffee Price: 55913.8671875


C:\Users\PC\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(


## RNN

In [10]:
# Use 'Giá Cà Phê' column as the target variable (coffee price to be predicted)
y = df['Giá Cà Phê'].values.reshape(-1, 1)

# Scale the target variable to the range [0, 1] (using MinMaxScaler)
scaler_y = MinMaxScaler()
y_scaled = scaler_y.fit_transform(y)

In [11]:
# Extract the gas price (diesel price) as an additional feature
X_gas = df['Dầu Diesel 1 lít'].values.reshape(-1, 1)

# Scale the gas price to the range [0, 1] (using MinMaxScaler)
scaler_gas = MinMaxScaler()
X_gas_scaled = scaler_gas.fit_transform(X_gas)


In [12]:
# Define the number of time steps for the RNN (adjusting to 11 for consistency)
time_steps = 11

# Create sequences for training the RNN
X_train, y_train = [], []
for i in range(time_steps, len(y_scaled)):
    X_train.append(np.concatenate((y_scaled[i-time_steps:i, 0], X_gas_scaled[i-time_steps:i, 0])))
    y_train.append(y_scaled[i, 0])

X_train, y_train = np.array(X_train), np.array(y_train)

# Reshape the input data for LSTM (number of samples, time steps, number of features)
X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))


In [14]:
# Build the RNN model
rnn_model = Sequential()
rnn_model.add(LSTM(units=50, return_sequences=True, input_shape=(X_train.shape[1], 1)))
rnn_model.add(LSTM(units=50))
rnn_model.add(Dense(units=1))

# Compile the RNN model
rnn_model.compile(optimizer='adam', loss='mean_squared_error')

# Train the RNN model
rnn_model.fit(X_train, y_train, epochs=100, batch_size=32)

Epoch 1/100
20/20 [==============================] - 3s 9ms/step - loss: 0.0340
Epoch 2/100
20/20 [==============================] - 0s 9ms/step - loss: 0.0156
Epoch 3/100
20/20 [==============================] - 0s 9ms/step - loss: 0.0099
Epoch 4/100
20/20 [==============================] - 0s 9ms/step - loss: 0.0039
Epoch 5/100
20/20 [==============================] - 0s 9ms/step - loss: 0.0018
Epoch 6/100
20/20 [==============================] - 0s 9ms/step - loss: 0.0016
Epoch 7/100
20/20 [==============================] - 0s 9ms/step - loss: 0.0014
Epoch 8/100
20/20 [==============================] - 0s 9ms/step - loss: 0.0014
Epoch 9/100
20/20 [==============================] - 0s 10ms/step - loss: 0.0015
Epoch 10/100
20/20 [==============================] - 0s 9ms/step - loss: 0.0013
Epoch 11/100
20/20 [==============================] - 0s 9ms/step - loss: 0.0013
Epoch 12/100
20/20 [==============================] - 0s 9ms/step - loss: 0.0018
Epoch 13/100
20/20 [================

20/20 [==============================] - 0s 9ms/step - loss: 6.4829e-04
Epoch 100/100
20/20 [==============================] - 0s 9ms/step - loss: 6.6417e-04


## Random forest

In [15]:
# Let's assume you have new diesel price and want to predict the coffee price
new_diesel_price = np.array([18.50]).reshape(-1, 1)

# Scale the new diesel price for the RNN model
new_diesel_scaled = scaler_gas.transform(new_diesel_price)

# Flatten the y_scaled array to ensure both arrays have the same number of dimensions
y_scaled_flatten = np.ravel(y_scaled[-time_steps:])

# Assuming new_diesel_scaled has the correct scaled gas price for prediction
X_pred_rnn = np.concatenate((y_scaled[-time_steps:], new_diesel_scaled)).reshape(1, -1, 1)

# Predict the scaled future coffee price using the RNN model
y_pred_scaled_rnn = rnn_model.predict(X_pred_rnn)

# Inverse scale the predicted coffee price to get the actual price using the RNN model
y_pred_rnn = scaler_y.inverse_transform(y_pred_scaled_rnn)

# Predict the coffee price using the Random Forest model
X_rf = df['Dầu Diesel 1 lít'].values.reshape(-1, 1)
rf_model = RandomForestRegressor()
rf_model.fit(X_rf, y)
y_pred_rf = rf_model.predict(new_diesel_price)


ValueError: in user code:

    File "C:\Users\PC\anaconda3\lib\site-packages\keras\engine\training.py", line 2169, in predict_function  *
        return step_function(self, iterator)
    File "C:\Users\PC\anaconda3\lib\site-packages\keras\engine\training.py", line 2155, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "C:\Users\PC\anaconda3\lib\site-packages\keras\engine\training.py", line 2143, in run_step  **
        outputs = model.predict_step(data)
    File "C:\Users\PC\anaconda3\lib\site-packages\keras\engine\training.py", line 2111, in predict_step
        return self(x, training=False)
    File "C:\Users\PC\anaconda3\lib\site-packages\keras\utils\traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "C:\Users\PC\anaconda3\lib\site-packages\keras\engine\input_spec.py", line 298, in assert_input_compatibility
        raise ValueError(

    ValueError: Input 0 of layer "seq

In [ ]:
# Get the dates for the actual data and the prediction
actual_dates = df['Ngày'].values[-len(y_scaled)+time_steps:]
predicted_date = df['Ngày'].values[-1] + pd.Timedelta(days=1)

# Plotting the actual and predicted coffee prices
plt.figure(figsize=(12, 6))
plt.plot(actual_dates, y[-len(y_scaled)+time_steps:], label='Actual Coffee Price')
plt.plot(predicted_date, y_pred_rf, 'ro', label='Predicted Coffee Price (Random Forest)')
plt.plot(predicted_date, y_pred_rnn[0, 0], 'go', label='Predicted Coffee Price (RNN)')
plt.xlabel('Date')
plt.ylabel('Coffee Price')
plt.title('Actual and Predicted Coffee Prices')
plt.legend()
plt.grid()
plt.show()